# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Load the csv exported in Part I to a DataFrame. WeatherDataCities.csv
weather_citi = "../output_data/WeatherDataCities.csv"

# # Read WeatherDataCities.csv and store into Pandas DataFrames
weather_cities = pd.read_csv(weather_citi)

weather_cities.head()

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Country,Wind Speed,Date
0,Poum,-20.23,164.02,81.32,83,100,NC,14.09,1609377964
1,Beringovskiy,63.05,179.32,4.26,93,94,RU,16.89,1609377965
2,Bambous Virieux,-20.34,57.76,77.00,88,40,MU,14.99,1609377967
3,Hithadhoo,-0.60,73.08,82.00,78,3,MV,9.08,1609377968
4,Bilibino,68.05,166.44,-32.35,80,99,RU,2.42,1609377969


In [20]:
weather_cities.dtypes

City             object
Lat             float64
Lng             float64
Max Temp (F)    float64
Humidity          int64
Cloudiness        int64
Country          object
Wind Speed      float64
Date              int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [22]:
# Store 'Lat' and 'Lng' into  locations as floats
locations = weather_cities[["Lat", "Lng"]].astype(float)
weights = weather_cities["Humidity"].astype(float)

In [23]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=80,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph and zero cloudiness.
fitting_weather = weather_cities.loc[(weather_cities['Humidity']>70) &
                                     (weather_cities['Humidity']<80) &
                                     (weather_cities['Wind Speed']<10) &
                                     (weather_cities['Cloudiness']==0)]

fitting_weather.shape[0]

8

In [25]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
fit_weather = fitting_weather.dropna()
fit_weather

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Country,Wind Speed,Date
7,Khovd,48.01,91.64,-5.94,78,0,MN,0.36,1609377972
89,Baykit,61.67,96.37,-25.06,78,0,RU,3.00,1609378073
246,Ji’an,27.12,114.98,35.60,73,0,CN,7.02,1609378282
257,Vila Velha,-20.33,-40.29,77.00,78,0,BR,8.05,1609378298
328,Bose,45.46,8.01,36.00,75,0,IT,3.98,1609378390
406,Esna,25.29,32.55,53.60,71,0,EG,4.70,1609378487
497,Morondava,-20.28,44.28,81.61,77,0,MG,7.65,1609378602
521,São Gonçalo,-22.82,-43.04,84.99,79,0,BR,2.24,1609378625


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# Store into variable named hotel_df.
hotel_df = fit_weather
hotel_name = ""

# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = hotel_name

hotel_df

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Country,Wind Speed,Date,Hotel Name
7,Khovd,48.01,91.64,-5.94,78,0,MN,0.36,1609377972,
89,Baykit,61.67,96.37,-25.06,78,0,RU,3.00,1609378073,
246,Ji’an,27.12,114.98,35.60,73,0,CN,7.02,1609378282,
257,Vila Velha,-20.33,-40.29,77.00,78,0,BR,8.05,1609378298,
328,Bose,45.46,8.01,36.00,75,0,IT,3.98,1609378390,
406,Esna,25.29,32.55,53.60,71,0,EG,4.70,1609378487,
497,Morondava,-20.28,44.28,81.61,77,0,MG,7.65,1609378602,
521,São Gonçalo,-22.82,-43.04,84.99,79,0,BR,2.24,1609378625,


In [27]:
# store geocoordinates
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary update each iteration
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and lng from hotel_df
    lat = row['Lat']
    lng = row['Lng']
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_names = requests.get(base_url, params=params)
    hotel_names = hotel_names.json()

    
#     print(json.dumps(hotel_names, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]
    except (KeyError, IndexError, TypeError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [28]:
hotel_names

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -22.8344912, 'lng': -43.0831877},
    'viewport': {'northeast': {'lat': -22.83324042010728,
      'lng': -43.08191542010728},
     'southwest': {'lat': -22.83594007989272, 'lng': -43.08461507989272}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Hotel Dom Quixote',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3096,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116731154595296910771">Alex Ramos</a>'],
     'photo_reference': 'ATtYBwLR9SlwgHNUOaLb7NFcKw4y0oWSLb2W5PiYk6jr_X4dWnqQ-I8dMeb4C9vQpxuGnklZWi20M8pMOevR9hmSNXXjbrJPP-hkxCUMrxG_s4Djg1OdQeglIgBzCOIErCG56MM73-xGcRWls7uzQdB7XhfPo_kZ5NsJN8dxOf-Kks9gT_Ft',
     'width': 4128}],
   'place_id': 'ChIJAaXIOk-bmQARCSPU3gaXLgg',
   'plus_code': {'compound_code': '5W88+6P Paraíso, São Gonçalo - State of Rio de Janeiro, Brazil',
    '

In [29]:
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Country,Wind Speed,Date,Hotel Name
7,Khovd,48.01,91.64,-5.94,78,0,MN,0.36,1609377972,Well hotel Khovd
89,Baykit,61.67,96.37,-25.06,78,0,RU,3.00,1609378073,
246,Ji’an,27.12,114.98,35.60,73,0,CN,7.02,1609378282,Ji'an Hotel
257,Vila Velha,-20.33,-40.29,77.00,78,0,BR,8.05,1609378298,Hotel Senac Ilha do Boi
328,Bose,45.46,8.01,36.00,75,0,IT,3.98,1609378390,Hotel Royal
406,Esna,25.29,32.55,53.60,71,0,EG,4.70,1609378487,Orient Star Dahabyia Isna 05 Nights Each Sunday
497,Morondava,-20.28,44.28,81.61,77,0,MG,7.65,1609378602,MAEVA HOTEL
521,São Gonçalo,-22.82,-43.04,84.99,79,0,BR,2.24,1609378625,Hotel Dom Quixote


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))